In [72]:
import urllib2, re
import sys,os
from firebase import firebase
import hashlib

In [73]:
from datetime import datetime

In [74]:
src_url = 'https://everettwa.gov/RSSFeed.aspx?ModID=58&CID=City-of-Everett-Public-Meetings-29'
req = urllib2.Request(src_url)
page = urllib2.urlopen(req).read()

# source html file
htmlFile = open('everettgov.html','wb')
htmlFile.write(page)
htmlFile.close()

p = re.compile(r'<title>(.*)</title>')
p2 = re.compile(r'<calendarEvent:EventDates>(.*)</calendarEvent:EventDates>')
p3 = re.compile(r'<calendarEvent:EventTimes>(.*)</calendarEvent:EventTimes>')
p4 = re.compile(r'<calendarEvent:Location>(.*)</calendarEvent:Location>')

t = p.findall(page)
d = p2.findall(page)
time = p3.findall(page)
L = p4.findall(page)



# clean data
cancel = []
titles = []
committees = []
for i in range(2,len(t)):
    if (t[i][0:8]=='CANCELED'):
        cancel.append("Yes")
    else :
        cancel.append("No")
    string = t[i].split("&#38;")
    titles.append(string[0])
    if (len(string)>1):
        committees.append(string[1])
    else:
        committees.append("")
        
dates = []
times = []
locations = []

for i in range(0,len(d)):
    s = d[i]
    s = " ".join(s.split(";br /&gt;"))
    s = ",".join(s.split(";br/&gt;"))
    s = "".join(s.split("&lt"))
    s = "".join(s.split("&gt;"))
    s = "".join(s.split("&amp;"))
    s = "".join(s.split("&quot;"))
    string = s.split(",")
    j = len(string)-1
    dateText = string[j-2][1:] + string[j-1]
    try:
        date = datetime.strptime(dateText,'%B %d %Y')
        dates.append(date)
    except:
        pass

for i in range(0,len(time)):
    s = time[i]
    s = " ".join(s.split(";br /&gt;"))
    s = ",".join(s.split(";br/&gt;"))
    s = "".join(s.split("&lt"))
    s = "".join(s.split("&gt;"))
    s = "".join(s.split("&amp;"))
    s = "".join(s.split("&quot;"))
    string = s.split("-")
    tt = string[0]
    times.append(tt)    
    
for i in range(0,len(L)):
    s = L[i]
    s = " ".join(s.split(";br /&gt;"))
    s = ",".join(s.split(";br/&gt;"))
    s = "".join(s.split("&lt"))
    s = "".join(s.split("&gt;"))
    s = "".join(s.split("&amp;"))
    s = "".join(s.split("&quot;"))
    s.replace(";",",")
    location = s
    locations.append(location)

# store in firebase
f = firebase.FirebaseApplication('https://wa-state-calendar.firebaseio.com')

# for i in range(len(titles)):
for i in range(100):
    agent = "Everett City Council"
    title = titles[i]
    canceled = cancel[i]
    committee = committees[i]
    date = dates[i]
    time = times[i]
    if len(time) > 10:
        time = "Flexible/On Appointment"
    location = locations[i]
    # use MD5 string as index to avoid duplicates
    tmpString = agent+title+committee+"Everett"+date.strftime('%m/%d/%Y')+time+location;
    MD5String = hashlib.md5(tmpString).hexdigest()
    
    MD5Write = open('MD5File.txt','a')
    if MD5String not in open('MD5File.txt').read():
        MD5Write.write(MD5String)
        MD5Write.write('\n')
        try:
            f.post('meeting/',
                {"MD5index":MD5String,
                 "title": title, 
                 "date": date,
                 "time": time,
                 "location": location,
                 "cancelOrNot":canceled,
                 "board": committee,
                 "city": "Everett",
                 "zipcode": 98201,
                 "description": "NA",
                 "contactName": "NA",
                 "contactEmail": "NA",
                 "contactPhone": "NA",
                 "weblink": "NA",
                 "summary": "NA"
                }
            )
        except:
            print "Firabase connection error."
        MD5Write.close()
    else:
        pass

# store data in a file        
events = []
for i in range(len(titles)):
   # item = titles[i]+' '+dates[i]+' '+times[i]+' '+zipcodes[i]+ ' '+locations[i]
    item = 'Everett City Council '+titles[i]+times[i]+' '+locations[i]
    events.append(item)
events.append(datetime.now().strftime("%I:%M %p on %B %d, %Y"))

webFile = open('crawler_everettgov.html','wb')
for event in events:
    webFile.write(event)
    webFile.write('\n')
webFile.close()


IndexError: list index out of range